## Join on enum and categorical columns
By the end of this lecture you will be able to:
- join on categorical columns
- join on enum columns
- assess whether joins on categorical or enum columns would be faster

I recommended that you do the lectures on "String and categorical dtypes" and "Categoricals and the string cache" in Section 3 before doing this lecture.

In [ ]:
import polars as pl
import numpy as np
np.random.seed(0)

We first create a short array with some integers

In [ ]:
integer_array = np.array([3,3,1,2])
integer_array

For the left `DataFrame` we convert each of the integers to an `id` string that starts with `"id"`. We keep the integers in the `values` column

In [ ]:
df_left = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integer_array],
            "values":integer_array
        }
    )
)
df_left

We then create the right `DataFrame` that has metadata about each `id`

In [ ]:
df_right = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(1,4)],
            "metadata":[i for i in range(1,4)]
        }
    )
)
df_right

When the `id` column is a string dtype we can join these `DataFrames` in the standard way 

In [ ]:
(
    df_left
    .join(
        df_right,
        on="id"
    )
)

> Polars cannot use a fast-track algorithm for joining string columns as the algorithm works on integers only

## Joins on categorical dtype
We cast the `id` column to categorical dtype

In [ ]:
df_left = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integer_array],
            "values":integer_array
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
)
df_left

And we cast the `id` column to categorical for the right `DataFrame`

In [ ]:
df_right = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(1,4)],
            "metadata":[i for i in range(1,4)]
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
)
df_right

If we try to join them on the categorical column we get a warning

In [ ]:
(
    
    df_left
    .join(
        df_right,
        on="id"
    )
)

We get a warning because we didn't cast to categorical for both `DataFrames` inside a `StringCache`:

As we are not inside a `StringCache` Polars doesn't know if the left and right `DataFrames` use the same mapping from strings to integers and so does a re-mapping. This re-mapping can be expensive for large `DataFrames` with many mappings.

We try casting to categorical again inside a `StringCache`

In [ ]:
with pl.StringCache():
    df_left = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in integer_array],
                "values":integer_array
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
    )
    
    df_right = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in range(1,4)],
                "metadata":[i for i in range(1,4)]
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
    )

We can now join the `DataFrames` in the standard way without a warning

In [ ]:
(
    
    df_left
    .join(
        df_right,
        on="id"
    )
)

We can also do the `join` or any other operations inside the `StringCache` block. 

## Joining `pl.Enum` columns
We can join `pl.Enum` columns without needing a `StringCache`.

We first define an instance of a `pl.Enum` dtype with the categories

In [ ]:
enum_dtype = pl.Enum(["id1","id2","id3"])

We then cast the `id` column to `pl.Enum` instead of `pl.Categorical` in the left and right `DataFrames`

In [ ]:
df_left_enum = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integer_array],
            "values":integer_array
        }
    )
    .with_columns(
        pl.col("id").cast(enum_dtype)
    )
)
df_left_enum

In [ ]:
df_right_enum = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(1,4)],
            "metadata":[i for i in range(1,4)]
        }
    )
    .with_columns(
        pl.col("id").cast(enum_dtype)
    )
)
df_right_enum

We can then go ahead and join these columns without needing a `StringCache` or triggering a re-mapping

In [ ]:
(
    df_left_enum
    .join(
        df_right_enum,
        on="id"
    )
)

## Faster joins?

In the exercises we compare the performance of joining `DataFrames` with:
- a string dtype
- a categorical dtype and
- an enum dtype

Overall we find that the categorical and enum dtypes are much faster. The key decisions are whether:
- the performance gain of categoricals/enums is worthwhile for your pipeline given the extra overhead in terms of working with the `StringCache`/creating the `pl.Enum`
- whether the categories found in your data are predictable enough for you to use the `pl.Enum` which is slightly easier to work with than categoricals and string caches

## Exercises
In the exercises you will develop your understanding of:
- joining on categorical columns
- joining on string columns
- assessing the performance difference of joins on categorical columns

## Exercise 1
The CITES and ISO CSV files are here 

In [ ]:
cites_csv_file = "../data/cites_extract.csv"
iso_csv_file = "../data/countries_extract.csv"

We want to join the ISO data for importers and exporters.

- create `DataFrames` from the CITES trade data and ISO country data in the following CSVs
- cast the join columns to categorical

Join the ISO data for both importers and exporters

We now do this join using a `pl.Enum`.

First create a `pl.Enum` that holds all of the 2-digit country codes - this requires data from both `DataFrames` (there are a variety of ways this can be done)

In [ ]:
countries_enum = pl.Enum(
<blank>
)

Do the first part of the left-join above of `df_ISO` to `df_CITES` with the join columns as `pl.Enum` dtypes

## Exercise 2

We compare the performance of sorted and unsorted joins on strings and categoricals. 

We create the left `DataFrame` with length `N` and random `id` strings in this function

In [ ]:
N = 1_000_000
# cardinality is number of unique values
cardinality = N // 2
def createLeftDataFrame(N:int,cardinality:int):
    """
    Create the left dataframe with columns:
    id - random strings of the form idX where X is between 0 and 0
    values - the integer X value
    physical - the physical integers underlying the categorical id column
    """
    # create the random integer array
    integer_array = np.random.randint(0,cardinality,N)
    return (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integer_array],
            "values":integer_array
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
    .with_columns(
        pl.col("id").to_physical().alias("physical")
    )
)
df_left = createLeftDataFrame(N = N,cardinality=cardinality)
df_left.head()

We create the right `DataFrame` with metadata about each `id` in this function

In [ ]:
def createRightDataFrame(N:int,cardinality:int):
    """
    Create the right dataframe with columns:
    id - the string ids covering the same range as the left dataframe
    meta - a metadata column that has the integer number from the id
    physical - the physical integers underlying the categorical id column
    """
    return (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(cardinality)],
            "meta":[i for i in range(cardinality)]
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
    .with_columns(
        pl.col("id").to_physical().alias("physical")
    )

)
df_right = createRightDataFrame(N = N,cardinality=cardinality)
df_right.head(3)

Create `df_left` and `df_right` inside a `StringCache`

In [ ]:
N = 10_000_000
cardinality = 10
<blank>

Time how long it takes to join on unsorted categorical columns

In [ ]:
%%timeit -n1 -r1 
(
    <blank>
)

Sort the categorical columns in new `DataFrames`

In [ ]:
df_left_sorted = <blank>
df_right_sorted = <blank>

Time how long it takes to join on sorted categorical columns (see the discussion on the results in the solutions)

Cast the categorical columns to strings in new `DataFrames`

In [ ]:
df_left_string = (df_left.<blank>)
df_right_string = (df_right.<blank>)

Time how long it takes to join on string columns

Now cast the `id` columns to a `pl.Enum` dtype

Time the join with the `pl.Enum` dtype

Do these comparisons again with higher cardinality e.g. `cardinality > 1000` and lower cardinality e.g. `cardinality = 10`

## Solutions

### Solution to Exercise 1

We want to join the ISO data for importers and exporters.

In a single query:
- create `DataFrames` from the CITES trade data and ISO country data in the following CSVs
- cast the relevant columns to categorical

In [ ]:
cites_csv_file = "../data/cites_extract.csv"
iso_csv_file = "../data/countries_extract.csv"

In [ ]:
with pl.StringCache():
    df_CITES = (
        pl.read_csv(cites_csv_file)
        .with_columns(
            pl.col("Importer").cast(pl.Categorical),
            pl.col("Exporter").cast(pl.Categorical),
        )
    )
    df_ISO = (
        pl.read_csv(iso_csv_file)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
    )

Join the ISO data for importers and exporters

In [ ]:
with pl.StringCache():
    df_CITES = (
        pl.read_csv(cites_csv_file)
        .with_columns(
            pl.col("Importer").cast(pl.Categorical),
            pl.col("Exporter").cast(pl.Categorical),
        )
    )
    df_ISO = (
        pl.read_csv(iso_csv_file)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
    )
    
(
    df_CITES
        .join(df_ISO,left_on="Importer",right_on="alpha-2", how="left",coalesce=True)
        .rename({"name":"name_importer","region":"region_importer"})
        .join(df_ISO,left_on="Exporter",right_on="alpha-2", how="left",coalesce=True)
        .rename({"name":"name_exporter","region":"region_exporter"})
)

Q: Could Polars do a fast-track join with `dfCites` on `Importer` if `set_sorted` was used?


In [ ]:
df_CITES["Importer"].is_sorted()

No, the column is not sorted

Do a fast-track join on the `Importer` and `Exporter` columns (copy your code from above)

See:
- the sorting on `df_ISO`
- the two sort calls on `df_CITES` in the join query

In [ ]:
with pl.StringCache():
    df_CITES = (
        pl.read_csv(cites_csv_file)
        .with_columns(
            pl.col("Importer").cast(pl.Categorical),
            pl.col("Exporter").cast(pl.Categorical),
        )
    )
    df_ISO = (
        pl.read_csv(iso_csv_file)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
        ### Sorting on df_ISO!
        .sort("alpha-2")
    )
(
    df_CITES
        .sort("Importer")
        .join(df_ISO,left_on="Importer",right_on="alpha-2", how="left",coalesce=True)
        .rename({"name":"name_importer","region":"region_importer"})
        .sort("Exporter")
        .join(df_ISO,left_on="Exporter",right_on="alpha-2", how="left",coalesce=True)
        .rename({"name":"name_exporter","region":"region_exporter"})
)

We now do this join using a `pl.Enum`.

First create a `pl.Enum` that holds all of the 2-digit country codes - this requires data from both `DataFrames`

In [ ]:
countries_enum = pl.Enum(
    list(
        set(
            df_ISO["alpha-2"].unique().to_list() + df_CITES["Importer"].unique().to_list())
    )
)
countries_enum

Do the first part of the left-join above of `df_ISO` to `df_CITES` with the join columns as `pl.Enum` dtypes

In [ ]:
(
    df_CITES
        .with_columns(
            pl.col("Importer").cast(countries_enum)
        )
        .sort("Importer")
        .join(
            df_ISO.with_columns(
                pl.col("alpha-2").cast(countries_enum)
            ),
            left_on="Importer",
            right_on="alpha-2", 
            how="left",
            coalesce=True
        )
        .rename({"name":"name_importer","region":"region_importer"})
        .sort("Exporter")
)

### Solution to Exercise 2

Create `df_left` and `df_right` inside a `StringCache`

In [ ]:
N = 100_000_000
cardinality = 100
with pl.StringCache():
    df_left = createLeftDataFrame(N = N,cardinality=cardinality)
    df_right = createRightDataFrame(N = N,cardinality=cardinality)

Time how long it takes to join on unsorted categorical columns

In [ ]:
%%timeit -n1 -r3
(
    df_left.join(df_right,on="id")
)

Sort the categorical columns

In [ ]:
df_left_sorted = df_left.sort("id")
df_right_sorted = df_right.sort("id")

Time how long it takes to join on sorted categorical columns

In [ ]:
%%timeit -n1 -r3
(
    df_left_sorted.join(df_right_sorted,on="id")
)

I get a **small** speed-up with the sorted data when cardinality is high. However, the relative performance varies from comparison-to-comparison and with different versions of Polars. Check this for your own data to see if you get a useful speed-up. 

Cast the categorical columns to strings and compare how long it takes to join on string columns 

In [ ]:
df_left_string = df_left.with_columns(pl.col("id").cast(pl.String))
df_right_string = df_right.with_columns(pl.col("id").cast(pl.String))

In [ ]:
%%timeit -n1 -r1
(
    df_left_string.join(df_right_string,on="id")
)

So the string join is indeed much slower.

Now cast the `id` columns to a `pl.Enum` dtype

In [ ]:
id_enum = pl.Enum(df_left["id"].cast(pl.String).unique())
df_left_enum = df_left.with_columns(pl.col("id").cast(pl.String).cast(id_enum))
df_right_enum = df_right.with_columns(pl.col("id").cast(pl.String).cast(id_enum))

Time the join with the `pl.Enum` dtype

In [ ]:
%%timeit -n1 -r3
(
    df_left_enum.join(df_right_enum,on="id")
)

Overall we find that:
- both categoricals and enum dtypes are much faster than string joins
- using sorted data has a very small impact
